In [1]:
import sys
sys.path.append('..')

In [2]:
import os
os.getcwd()

'/mnt/3B1D7BDC2D2641B0/Documents/GIT/SQUAD_NLP/pythonbooks'

In [3]:
os.chdir('..')
os.getcwd()

'/mnt/3B1D7BDC2D2641B0/Documents/GIT/SQUAD_NLP'

In [4]:
%env CUDA_LAUNCH_BLOCKING=1

env: CUDA_LAUNCH_BLOCKING=1


# Хочется вытащить все в питонбук, потому что из консоли работать неудобно

# Обучение

In [5]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as sched
import torch.utils.data as data
import util

from args import get_train_args
from collections import OrderedDict
from json import dumps
from models import BiDAF, BiDAF_CBE
from models_franky import BiDAF_franky
from tensorboardX import SummaryWriter
from tqdm import tqdm
from ujson import load as json_load
from util import collate_fn, SQuAD

In [6]:
train_args = type('', (), {})() # empty "structure" where you can add fields

train_args.train_record_file = './data/train.npz'
train_args.dev_record_file = './data/dev.npz'
train_args.test_record_file = './data/test.npz'
train_args.word_emb_file = './data/word_emb.json'
train_args.char_emb_file = './data/char_emb.json'
train_args.train_eval_file = './data/train_eval.json'
train_args.dev_eval_file = './data/dev_eval.json'
train_args.test_eval_file = './data/test_eval.json'

train_args.name = 'train'       # ???
train_args.max_ans_len = 15     # Maximum length of a predicted answer.
train_args.num_workers = 6      # Number of sub-processes to use per data loader.
train_args.save_dir = './save/' # Base directory for saving information.
train_args.batch_size = 80     # Batch size per GPU. Scales automatically when \
                                #    multiple GPUs are available.
train_args.use_squad_v2 = True  # Whether to use SQuAD 2.0 (unanswerable) questions.
train_args.hidden_size = 100    # Number of features in encoder hidden layers
train_args.num_visuals = 10     # Number of examples to visualize in TensorBoard.
train_args.load_path = None     # Path to load as a model checkpoint.

train_args.eval_steps = 25000   # Number of steps between successive evaluations
train_args.lr = 1.0             # learning rate
train_args.l2_wd = 0.0          # L2 weight decay.
train_args.num_epochs = 30      # Number of epochs for which to train. Negative means forever.
train_args.drop_prob = 0.2      # Probability of zeroing an activation in dropout layers.
train_args.metric_name = 'F1'   # choices=('NLL', 'EM', 'F1'), name of dev metric to determine best checkpoint.
train_args.max_checkpoints = 5  # Maximum number of checkpoints to keep on disk.
train_args.max_grad_norm = 5.0  # Maximum gradient norm for gradient clipping.
train_args.seed = 224           # Random seed
train_args.ema_decay = 0.999    # Decay rate for exponential moving average of parameters.

if train_args.metric_name == 'NLL':
    # Best checkpoint is the one that minimizes negative log-likelihood
    train_args.maximize_metric = False
elif train_args.metric_name in ('EM', 'F1'):
    # Best checkpoint is the one that maximizes EM or F1
    train_args.maximize_metric = True

In [7]:
# from train.py

lambda3 = lambda step: (0.01 + 0.001 * (step+1)) if step < 1000 else 1.0 if step < 2500 else 0.8 * 0.99989 ** (step - 2500)

In [8]:
for i in range(45500):
    print(i, ' = ', lambda3(i))

0  =  0.011
1  =  0.012
2  =  0.013000000000000001
3  =  0.014
4  =  0.015
5  =  0.016
6  =  0.017
7  =  0.018000000000000002
8  =  0.019000000000000003
9  =  0.02
10  =  0.020999999999999998
11  =  0.022
12  =  0.023
13  =  0.024
14  =  0.025
15  =  0.026000000000000002
16  =  0.027000000000000003
17  =  0.028000000000000004
18  =  0.028999999999999998
19  =  0.03
20  =  0.031
21  =  0.032
22  =  0.033
23  =  0.034
24  =  0.035
25  =  0.036000000000000004
26  =  0.037
27  =  0.038
28  =  0.039
29  =  0.04
30  =  0.041
31  =  0.042
32  =  0.043000000000000003
33  =  0.044000000000000004
34  =  0.045000000000000005
35  =  0.046000000000000006
36  =  0.047
37  =  0.048
38  =  0.049
39  =  0.05
40  =  0.051000000000000004
41  =  0.052000000000000005
42  =  0.053000000000000005
43  =  0.054
44  =  0.055
45  =  0.056
46  =  0.057
47  =  0.058
48  =  0.059000000000000004
49  =  0.060000000000000005
50  =  0.061000000000000006
51  =  0.062000000000000006
52  =  0.063
53  =  0.064
54  =  0.065

1314  =  1.0
1315  =  1.0
1316  =  1.0
1317  =  1.0
1318  =  1.0
1319  =  1.0
1320  =  1.0
1321  =  1.0
1322  =  1.0
1323  =  1.0
1324  =  1.0
1325  =  1.0
1326  =  1.0
1327  =  1.0
1328  =  1.0
1329  =  1.0
1330  =  1.0
1331  =  1.0
1332  =  1.0
1333  =  1.0
1334  =  1.0
1335  =  1.0
1336  =  1.0
1337  =  1.0
1338  =  1.0
1339  =  1.0
1340  =  1.0
1341  =  1.0
1342  =  1.0
1343  =  1.0
1344  =  1.0
1345  =  1.0
1346  =  1.0
1347  =  1.0
1348  =  1.0
1349  =  1.0
1350  =  1.0
1351  =  1.0
1352  =  1.0
1353  =  1.0
1354  =  1.0
1355  =  1.0
1356  =  1.0
1357  =  1.0
1358  =  1.0
1359  =  1.0
1360  =  1.0
1361  =  1.0
1362  =  1.0
1363  =  1.0
1364  =  1.0
1365  =  1.0
1366  =  1.0
1367  =  1.0
1368  =  1.0
1369  =  1.0
1370  =  1.0
1371  =  1.0
1372  =  1.0
1373  =  1.0
1374  =  1.0
1375  =  1.0
1376  =  1.0
1377  =  1.0
1378  =  1.0
1379  =  1.0
1380  =  1.0
1381  =  1.0
1382  =  1.0
1383  =  1.0
1384  =  1.0
1385  =  1.0
1386  =  1.0
1387  =  1.0
1388  =  1.0
1389  =  1.0
1390  =  1.0

2647  =  0.7871673259856462
2648  =  0.7870807375797878
2649  =  0.786994158698654
2650  =  0.7869075893411971
2651  =  0.7868210295063696
2652  =  0.7867344791931238
2653  =  0.7866479384004125
2654  =  0.7865614071271884
2655  =  0.7864748853724044
2656  =  0.7863883731350133
2657  =  0.7863018704139685
2658  =  0.7862153772082229
2659  =  0.78612889351673
2660  =  0.7860424193384431
2661  =  0.7859559546723158
2662  =  0.7858694995173018
2663  =  0.7857830538723549
2664  =  0.7856966177364288
2665  =  0.7856101911084777
2666  =  0.7855237739874559
2667  =  0.7854373663723172
2668  =  0.7853509682620161
2669  =  0.7852645796555073
2670  =  0.7851782005517451
2671  =  0.7850918309496844
2672  =  0.7850054708482799
2673  =  0.7849191202464865
2674  =  0.7848327791432594
2675  =  0.7847464475375536
2676  =  0.7846601254283244
2677  =  0.7845738128145272
2678  =  0.7844875096951176
2679  =  0.7844012160690511
2680  =  0.7843149319352835
2681  =  0.7842286572927706
2682  =  0.784142392140

4147  =  0.6674281623846239
4148  =  0.6673547452867616
4149  =  0.66728133626478
4150  =  0.6672079353177908
4151  =  0.6671345424449058
4152  =  0.6670611576452368
4153  =  0.6669877809178959
4154  =  0.6669144122619949
4155  =  0.666841051676646
4156  =  0.6667676991609616
4157  =  0.6666943547140538
4158  =  0.6666210183350352
4159  =  0.6665476900230183
4160  =  0.6664743697771157
4161  =  0.6664010575964402
4162  =  0.6663277534801045
4163  =  0.6662544574272218
4164  =  0.6661811694369046
4165  =  0.6661078895082666
4166  =  0.6660346176404206
4167  =  0.6659613538324801
4168  =  0.6658880980835585
4169  =  0.6658148503927693
4170  =  0.665741610759226
4171  =  0.6656683791820425
4172  =  0.6655951556603324
4173  =  0.6655219401932098
4174  =  0.6654487327797884
4175  =  0.6653755334191827
4176  =  0.6653023421105065
4177  =  0.6652291588528744
4178  =  0.6651559836454005
4179  =  0.6650828164871995
4180  =  0.6650096573773858
4181  =  0.6649365063150743
4182  =  0.6648633632993

5314  =  0.5870175409321504
5315  =  0.5869529690026479
5316  =  0.5868884041760575
5317  =  0.5868238464515981
5318  =  0.5867592958284885
5319  =  0.5866947523059473
5320  =  0.5866302158831936
5321  =  0.5865656865594464
5322  =  0.5865011643339249
5323  =  0.586436649205848
5324  =  0.5863721411744355
5325  =  0.5863076402389062
5326  =  0.58624314639848
5327  =  0.586178659652376
5328  =  0.5861141799998143
5329  =  0.5860497074400142
5330  =  0.5859852419721958
5331  =  0.5859207835955788
5332  =  0.5858563323093833
5333  =  0.5857918881128292
5334  =  0.5857274510051368
5335  =  0.5856630209855261
5336  =  0.5855985980532177
5337  =  0.5855341822074318
5338  =  0.5854697734473889
5339  =  0.5854053717723098
5340  =  0.5853409771814148
5341  =  0.5852765896739247
5342  =  0.5852122092490606
5343  =  0.5851478359060431
5344  =  0.5850834696440934
5345  =  0.5850191104624326
5346  =  0.5849547583602818
5347  =  0.5848904133368621
5348  =  0.5848260753913949
5349  =  0.5847617445231

...

38143  =  0.015857872610832633
38144  =  0.015856128244845443
38145  =  0.015854384070738508
38146  =  0.015852640088490726
38147  =  0.01585089629808099
38148  =  0.015849152699488202
38149  =  0.015847409292691256
38150  =  0.01584566607766906
38151  =  0.015843923054400515
38152  =  0.01584218022286453
38153  =  0.015840437583040014
38154  =  0.015838695134905878
38155  =  0.01583695287844104
38156  =  0.01583521081362441
38157  =  0.01583346894043491
38158  =  0.01583172725885146
38159  =  0.015829985768852986
38160  =  0.015828244470418414
38161  =  0.015826503363526666
38162  =  0.015824762448156678
38163  =  0.01582302172428738
38164  =  0.015821281191897706
38165  =  0.015819540850966596
38166  =  0.01581780070147299
38167  =  0.015816060743395824
38168  =  0.01581432097671405
38169  =  0.015812581401406612
38170  =  0.015810842017452458
38171  =  0.015809102824830536
38172  =  0.015807363823519802
38173  =  0.015805625013499216
38174  =  0.01580388639474773
38175  =  0.0158021

39809  =  0.013202365151183016
39810  =  0.013200912891016384
39811  =  0.013199460790598373
39812  =  0.013198008849911406
39813  =  0.013196557068937916
39814  =  0.013195105447660331
39815  =  0.013193653986061087
39816  =  0.01319220268412262
39817  =  0.013190751541827365
39818  =  0.013189300559157763
39819  =  0.013187849736096255
39820  =  0.013186399072625282
39821  =  0.013184948568727296
39822  =  0.013183498224384733
39823  =  0.01318204803958005
39824  =  0.013180598014295698
39825  =  0.013179148148514123
39826  =  0.013177698442217787
39827  =  0.01317624889538914
39828  =  0.013174799508010647
39829  =  0.013173350280064767
39830  =  0.013171901211533957
39831  =  0.013170452302400687
39832  =  0.013169003552647425
39833  =  0.013167554962256632
39834  =  0.013166106531210784
39835  =  0.013164658259492349
39836  =  0.013163210147083805
39837  =  0.013161762193967625
39838  =  0.013160314400126286
39839  =  0.013158866765542272
39840  =  0.013157419290198062
39841  =  0

41476  =  0.010990331212340746
41477  =  0.010989122275907388
41478  =  0.010987913472457037
41479  =  0.010986704801975066
41480  =  0.010985496264446849
41481  =  0.010984287859857758
41482  =  0.010983079588193174
41483  =  0.010981871449438472
41484  =  0.010980663443579033
41485  =  0.01097945557060024
41486  =  0.010978247830487472
41487  =  0.010977040223226119
41488  =  0.010975832748801562
41489  =  0.010974625407199193
41490  =  0.010973418198404401
41491  =  0.010972211122402577
41492  =  0.010971004179179112
41493  =  0.0109697973687194
41494  =  0.01096859069100884
41495  =  0.010967384146032828
41496  =  0.010966177733776766
41497  =  0.01096497145422605
41498  =  0.010963765307366084
41499  =  0.010962559293182274
41500  =  0.010961353411660022
41501  =  0.01096014766278474
41502  =  0.010958942046541832
41503  =  0.010957736562916712
41504  =  0.010956531211894792
41505  =  0.010955325993461482
41506  =  0.0109541209076022
41507  =  0.010952915954302363
41508  =  0.0109

42976  =  0.009318548068374366
42977  =  0.009317523028086845
42978  =  0.009316498100553756
42979  =  0.009315473285762694
42980  =  0.009314448583701259
42981  =  0.009313423994357051
42982  =  0.009312399517717672
42983  =  0.009311375153770722
42984  =  0.009310350902503808
42985  =  0.009309326763904531
42986  =  0.009308302737960501
42987  =  0.009307278824659326
42988  =  0.009306255023988613
42989  =  0.009305231335935972
42990  =  0.009304207760489019
42991  =  0.009303184297635365
42992  =  0.009302160947362624
42993  =  0.009301137709658415
42994  =  0.009300114584510352
42995  =  0.009299091571906054
42996  =  0.009298068671833144
42997  =  0.009297045884279242
42998  =  0.00929602320923197
42999  =  0.009295000646678955
43000  =  0.00929397819660782
43001  =  0.009292955859006193
43002  =  0.0092919336338617
43003  =  0.009290911521161975
43004  =  0.009289889520894648
43005  =  0.009288867633047348
43006  =  0.009287845857607713
43007  =  0.009286824194563377
43008  =  0.

44309  =  0.008047558589828056
44310  =  0.008046673358383173
44311  =  0.008045788224313751
44312  =  0.008044903187609077
44313  =  0.00804401824825844
44314  =  0.00804313340625113
44315  =  0.008042248661576441
44316  =  0.008041364014223668
44317  =  0.008040479464182104
44318  =  0.008039595011441043
44319  =  0.008038710655989784
44320  =  0.008037826397817624
44321  =  0.008036942236913864
44322  =  0.008036058173267803
44323  =  0.008035174206868742
44324  =  0.008034290337705986
44325  =  0.00803340656576884
44326  =  0.008032522891046603
44327  =  0.008031639313528589
44328  =  0.0080307558332041
44329  =  0.008029872450062446
44330  =  0.00802898916409294
44331  =  0.008028105975284888
44332  =  0.008027222883627607
44333  =  0.008026339889110409
44334  =  0.008025456991722605
44335  =  0.008024574191453516
44336  =  0.008023691488292455
44337  =  0.008022808882228743
44338  =  0.008021926373251697
44339  =  0.008021043961350638
44340  =  0.00802016164651489
44341  =  0.008

In [9]:
def main(args):
    # Set up logging and devices
    args.save_dir = util.get_save_dir(args.save_dir, args.name, training=True)
    log = util.get_logger(args.save_dir, args.name)
    tbx = SummaryWriter(args.save_dir)
    device, args.gpu_ids = util.get_available_devices()
    log.info('Args: {}'.format(dumps(vars(args), indent=4, sort_keys=True)))
    args.batch_size *= max(1, len(args.gpu_ids))

    # Set random seed
    log.info('Using random seed {}...'.format(args.seed))
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)

    # Get embeddings
    log.info('Loading embeddings...')
    word_vectors = util.torch_from_json(args.word_emb_file)

    
    
    # Get data loader
    log.info('Building dataset...')
    train_dataset = SQuAD(args.train_record_file, args.use_squad_v2)
    train_loader = data.DataLoader(train_dataset,
                                   batch_size=args.batch_size,
                                   shuffle=True,
                                   num_workers=args.num_workers,
                                   collate_fn=collate_fn)
    dev_dataset = SQuAD(args.dev_record_file, args.use_squad_v2)
    dev_loader = data.DataLoader(dev_dataset,
                                 batch_size=args.batch_size,
                                 shuffle=False,
                                 num_workers=args.num_workers,
                                 collate_fn=collate_fn)
    
    
    # Get model
    log.info('Building model...')
    model = BiDAF_franky(word_vectors=word_vectors,
                      hidden_size=args.hidden_size,
                      drop_prob=args.drop_prob,
                      char_dict_size=1400,#train_dataset.get_char_dict_size(),
                      char_emb_size=64,
                      conv_kernel_size=3,
                      conv_depth1=50,
                      conv_output_hidden_size=args.hidden_size)
#     model.summary()
    model = nn.DataParallel(model, args.gpu_ids)
    if args.load_path:
        log.info('Loading checkpoint from {}...'.format(args.load_path))
        model, step = util.load_model(model, args.load_path, args.gpu_ids)
    else:
        step = 0
    model = model.to(device)
    model.train()
   
    ema = util.EMA(model, args.ema_decay)

    # Get saver
    saver = util.CheckpointSaver(args.save_dir,
                                 max_checkpoints=args.max_checkpoints,
                                 metric_name=args.metric_name,
                                 maximize_metric=args.maximize_metric,
                                 log=log)

    # Get optimizer and scheduler
    optimizer = optim.Adadelta(model.parameters(), args.lr,
                               weight_decay=args.l2_wd)
    scheduler = sched.LambdaLR(optimizer, lambda3)  # Constant LR

    

    # Train
    log.info('Training...')
    steps_till_eval = args.eval_steps
    steps_till_print_lr = 5000
    epoch = step // len(train_dataset)
    while epoch != args.num_epochs:
        epoch += 1
        log.info('Starting epoch {}...'.format(epoch))
        with torch.enable_grad(), \
                tqdm(total=len(train_loader.dataset)) as progress_bar:
            for cw_idxs, cc_idxs, qw_idxs, qc_idxs, y1, y2, ids in train_loader:
                # Setup for forward
                cw_idxs = cw_idxs.to(device)
                cc_idxs = cc_idxs.to(device)
                qw_idxs = qw_idxs.to(device)
                qc_idxs = qc_idxs.to(device)
                batch_size = cw_idxs.size(0)
                optimizer.zero_grad()

                # Forward
                log_p1, log_p2 = model(cw_idxs, cc_idxs, qw_idxs, qc_idxs)
                y1, y2 = y1.to(device), y2.to(device)
                loss = F.nll_loss(log_p1, y1) + F.nll_loss(log_p2, y2)
                loss_val = loss.item()

                # Backward
                loss.backward()
                nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()
                
                
                ema(model, step // batch_size)

                # Log info
                step += batch_size
                progress_bar.update(batch_size)
                progress_bar.set_postfix(epoch=epoch,
                                         NLL=loss_val)
                tbx.add_scalar('train/NLL', loss_val, step)
                tbx.add_scalar('train/LR',
                               optimizer.param_groups[0]['lr'],
                               step)

                steps_till_eval -= batch_size
                
                steps_till_print_lr -= batch_size
                if steps_till_print_lr <= 0:
                    print('Learning rate: ', end='')
                    for param_group in optimizer.param_groups:
                        print(param_group['lr'])
                    steps_till_print_lr = 5000
                if steps_till_eval <= 0:
                    steps_till_eval = args.eval_steps

                    # Evaluate and save checkpoint
                    log.info('Evaluating at step {}...'.format(step))
                    #ema.assign(model)
                    results, pred_dict = evaluate(model, dev_loader, device,
                                                  args.dev_eval_file,
                                                  args.max_ans_len,
                                                  args.use_squad_v2)
                    saver.save(step, model, results[args.metric_name], device)
                    #ema.resume(model)

                    # Log to console
                    results_str = ', '.join('{}: {:05.2f}'.format(k, v)
                                            for k, v in results.items())
                    log.info('Dev {}'.format(results_str))

                    # Log to TensorBoard
                    log.info('Visualizing in TensorBoard...')
                    for k, v in results.items():
                        tbx.add_scalar('dev/{}'.format(k), v, step)
                    util.visualize(tbx,
                                   pred_dict=pred_dict,
                                   eval_path=args.dev_eval_file,
                                   step=step,
                                   split='dev',
                                   num_visuals=args.num_visuals)

In [10]:
def evaluate(model, data_loader, device, eval_file, max_len, use_squad_v2):
    nll_meter = util.AverageMeter()

    model.eval()
    pred_dict = {}
    with open(eval_file, 'r') as fh:
        gold_dict = json_load(fh)
    with torch.no_grad(), \
            tqdm(total=len(data_loader.dataset)) as progress_bar:
        for cw_idxs, cc_idxs, qw_idxs, qc_idxs, y1, y2, ids in data_loader:
            # Setup for forward
            cw_idxs = cw_idxs.to(device)
            cc_idxs = cc_idxs.to(device)
            qw_idxs = qw_idxs.to(device)
            qc_idxs = qc_idxs.to(device)
            batch_size = cw_idxs.size(0)

            # Forward
            log_p1, log_p2 = model(cw_idxs, cc_idxs, qw_idxs, qc_idxs)
            y1, y2 = y1.to(device), y2.to(device)
            loss = F.nll_loss(log_p1, y1) + F.nll_loss(log_p2, y2)
            nll_meter.update(loss.item(), batch_size)

            # Get F1 and EM scores
            p1, p2 = log_p1.exp(), log_p2.exp()
            starts, ends = util.discretize(p1, p2, max_len, use_squad_v2)

            # Log info
            progress_bar.update(batch_size)
            progress_bar.set_postfix(NLL=nll_meter.avg)

            preds, _ = util.convert_tokens(gold_dict,
                                           ids.tolist(),
                                           starts.tolist(),
                                           ends.tolist(),
                                           use_squad_v2)
            pred_dict.update(preds)

    model.train()

    results = util.eval_dicts(gold_dict, pred_dict, use_squad_v2)
    results_list = [('NLL', nll_meter.avg),
                    ('F1', results['F1']),
                    ('EM', results['EM'])]
    if use_squad_v2:
        results_list.append(('AvNA', results['AvNA']))
    results = OrderedDict(results_list)

    return results, pred_dict

In [11]:
main(train_args)

[05.16.19 02:28:20] Args: {
    "batch_size": 80,
    "char_emb_file": "./data/char_emb.json",
    "dev_eval_file": "./data/dev_eval.json",
    "dev_record_file": "./data/dev.npz",
    "drop_prob": 0.2,
    "ema_decay": 0.999,
    "eval_steps": 25000,
    "gpu_ids": [
        0
    ],
    "hidden_size": 100,
    "l2_wd": 0.0,
    "load_path": null,
    "lr": 1.0,
    "max_ans_len": 15,
    "max_checkpoints": 5,
    "max_grad_norm": 5.0,
    "maximize_metric": true,
    "metric_name": "F1",
    "name": "train",
    "num_epochs": 30,
    "num_visuals": 10,
    "num_workers": 6,
    "save_dir": "./save/train/train-68",
    "seed": 224,
    "test_eval_file": "./data/test_eval.json",
    "test_record_file": "./data/test.npz",
    "train_eval_file": "./data/train_eval.json",
    "train_record_file": "./data/train.npz",
    "use_squad_v2": true,
    "word_emb_file": "./data/word_emb.json"
}
[05.16.19 02:28:20] Using random seed 224...
[05.16.19 02:28:20] Loading embeddings...
[05.16.19 02:28:

  0%|          | 0/129922 [00:00<?, ?it/s]

[05.16.19 02:29:27] Saver will maximize F1...
[05.16.19 02:29:27] Training...
[05.16.19 02:29:27] Starting epoch 1...


  4%|▍         | 5040/129922 [01:27<38:04, 54.67it/s, NLL=8.13, epoch=1] 

Learning rate: 0.073


...


 83%|████████▎ | 4960/5951 [00:13<00:02, 351.05it/s, NLL=2.7]


 56%|█████▋    | 73280/129922 [19:42<14:10, 66.59it/s, NLL=1.94, epoch=12]

[05.16.19 09:20:01] Saved checkpoint: ./save/train/train-68/step_1502422.pth.tar
[05.16.19 09:20:01] Removed checkpoint: ./save/train/train-68/step_1502422.pth.tar
[05.16.19 09:20:01] Dev NLL: 02.80, F1: 62.74, EM: 59.30, AvNA: 69.25
[05.16.19 09:20:01] Visualizing in TensorBoard...


 59%|█████▊    | 76240/129922 [20:28<13:07, 68.20it/s, NLL=2.28, epoch=12]  
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/celidos/anaconda3/envs/venv_tensorflowgpu/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/home/celidos/anaconda3/envs/venv_tensorflowgpu/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/celidos/anaconda3/envs/venv_tensorflowgpu/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/home/celidos/anaconda3/envs/venv_tensorflowgpu/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/celidos/anaconda3/envs/venv_tensorflowgpu/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/home/celidos/anaconda3/envs/venv_tensorflowgpu/lib/python3.6/multiproc

KeyboardInterrupt: 